# Datacamp Nobel Prize analysis

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Importing the data
data = pd.read_csv('data/nobel.csv')

# Exploring the data
print(f'Data columns: {data.columns} \n\n')
print(f'Data basic information: {data.info()}\n\n')
print(f'Data sample rows: {data.head(3)}')

Data columns: Index(['year', 'category', 'prize', 'motivation', 'prize_share', 'laureate_id',
       'laureate_type', 'full_name', 'birth_date', 'birth_city',
       'birth_country', 'sex', 'organization_name', 'organization_city',
       'organization_country', 'death_date', 'death_city', 'death_country'],
      dtype='object') 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   year                  1000 non-null   int64 
 1   category              1000 non-null   object
 2   prize                 1000 non-null   object
 3   motivation            912 non-null    object
 4   prize_share           1000 non-null   object
 5   laureate_id           1000 non-null   int64 
 6   laureate_type         1000 non-null   object
 7   full_name             1000 non-null   object
 8   birth_date            968 non-null    object
 9   birth

In [3]:
# Fixing the categories
# data['year'] = data['year'].astype('category')
data['category'] = data['category'].astype('category')
data['prize_share'] = data['prize_share'].astype('category')
data['laureate_type'] = data['laureate_type'].astype('category')
data['sex'] = data['sex'].astype('category')

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   year                  1000 non-null   int64   
 1   category              1000 non-null   category
 2   prize                 1000 non-null   object  
 3   motivation            912 non-null    object  
 4   prize_share           1000 non-null   category
 5   laureate_id           1000 non-null   int64   
 6   laureate_type         1000 non-null   category
 7   full_name             1000 non-null   object  
 8   birth_date            968 non-null    object  
 9   birth_city            964 non-null    object  
 10  birth_country         969 non-null    object  
 11  sex                   970 non-null    category
 12  organization_name     736 non-null    object  
 13  organization_city     735 non-null    object  
 14  organization_country  735 non-null    object  
 15  death

In [4]:
# Question 1

top_gender = data.groupby('sex')['laureate_id'].agg('count').sort_values(ascending=False).index[0]
print(top_gender)

top_country = data.groupby('birth_country')['laureate_id'].agg('count').sort_values(ascending=False).index[0]
print(top_country)

Male
United States of America


In [5]:
# Question 2
# Finding the decade with the highest number of USA awards

data['decade'] = (data.year/10).astype(int)*10
usa = data[data.birth_country == 'United States of America']
max_decade_usa = data.groupby('decade')['laureate_id'].agg('count').sort_values(ascending=False).index[0]
print(max_decade_usa)

2000


In [29]:
# Question 3
# Decade with  the highest female proportion

sex_decade = data.pivot_table(index=['decade', 'category'], values='laureate_id', columns='sex', aggfunc=len, fill_value=0)
answer = sex_decade[sex_decade.Female == sex_decade.Female.max()].index[0]
max_female_dict = {}
max_female_dict[answer[0]] = answer[1]